<a href="https://colab.research.google.com/github/indl1670/KST_Project/blob/main/1cycle_sideproject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/MyDrive/kst_project/oneCycle

/content/drive/MyDrive/kst_project/oneCycle/yolov5


In [ ]:
git clone https://github.com/ultralytics/yolov5.git

In [ ]:
cd yolov5

In [ ]:
!pip install -r requirements.txt
!pip install wandb

In [13]:
import os

dataset_dir = '/content/drive/Shareddrives/KST Project #2/split_image'
dataset_folder = os.listdir(dataset_dir)

# 수량 일치 확인
for dataset in dataset_folder:
  print(str(dataset))
  img, lab = os.listdir(os.path.join(dataset_dir, dataset))
  if lab == 'images': # 폴더 순서 설정
    lab = 'labels'
    img = 'images'
    
  img_list = os.listdir(os.path.join(dataset_dir, dataset, img))
  lab_list = os.listdir(os.path.join(dataset_dir, dataset, lab))
  cnt=0
  for l in lab_list:
    l2i = l[:-3] + 'jpg'
    if l2i in img_list:
      cnt+=1
  print('img: ', len(img_list))
  print('lab: ', len(lab_list))
  print('match: ', cnt)


train
img:  987
lab:  987
match:  987
valid
img:  253
lab:  253
match:  253
test
img:  170
lab:  170
match:  170


In [7]:
!python train.py --img 640 --batch 16 --epochs 150 --data ./data/custom.yaml --cfg ./models/yolov5s.yaml --weights yolov5s.pt

wandb: Currently logged in as: heoseokyong (use `wandb login --relogin` to force relogin)
train: weights=yolov5s.pt, cfg=./models/yolov5s.yaml, data=./data/custom.yaml, hyp=data/hyps/hyp.scratch.yaml, epochs=150, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=0, save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: ⚠️ YOLOv5 is out of date by 3 commits. Use `git pull` or `git clone https://github.com/ultralytics/yolov5` to update.
YOLOv5 🚀 2021-12-7 torch 1.10.0+cu111 CUDA:0 (Tesla K80, 11441MiB)

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0